In [1]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
import scipy.io as sio
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn import plotting
from PyPDF2 import PdfFileMerger
from natsort import natsorted
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA

In [2]:
ukb_smri_data_path = "/Users/xli77/Documents/MIVA/output/UKB_MMIVA_C30_preregSite_SMRI_MancovanOuts_wX_FINAL.mat"
ukb_smri_data = sio.loadmat(ukb_smri_data_path)['MODELUKB0s_ful']
ukb_smri_data_array = ukb_smri_data[0][0][0]
ukb_smri_data_key = np.squeeze(ukb_smri_data[0][0][3])
age_idx = np.where(ukb_smri_data_key==['age_when_attended_assessment_centre_f21003_2_0'])[0][0]
sex_idx = np.where(ukb_smri_data_key==['sex_f31_0_0'])[0][0]
age = ukb_smri_data_array[:, age_idx]
sex = ukb_smri_data_array[:, sex_idx]
age_sex = age * sex

In [3]:
datapath="/Users/xli77/Documents/MISA/results/SIVA/fixedSubspace/um2mm/"
subspace_struct_list=['234111','2222211','333111','441111']

Y = np.zeros((4,2,2,12,2907)) # S1-4, UA/MSIVA, M1-2, voxel, source

num_iter=21
corr = np.zeros((4,6,12,12))

for i,ss in enumerate(subspace_struct_list):

    data=sio.loadmat(os.path.join(datapath,f"subspace_struct_{ss}","um_neuroimaging_Y.mat"))
    Y1=np.squeeze(data['Y1'])

    data=sio.loadmat(os.path.join(datapath,f"subspace_struct_{ss}","ummm_neuroimaging_Y.mat"))
    Y2=np.squeeze(data['Y2'])

    Y[i,0,0]=Y1[0]
    Y[i,0,1]=Y1[1]
    Y[i,1,0]=Y2[0]
    Y[i,1,1]=Y2[1]

In [4]:
# S2 subspace across two modalities
predictor = np.concatenate((np.expand_dims(age,axis=1),np.expand_dims(sex,axis=1),np.expand_dims(age_sex,axis=1)), axis=1)
response = np.concatenate((Y[1,0,0,:], Y[1,0,1,:]), axis=0).T
print(predictor.shape, response.shape)

(2907, 3) (2907, 24)


In [6]:
data = np.concatenate((predictor, response), axis=1)

data_name = ['Age', 'Sex', 'Age_Sex']
for i in range(2):
    for j in range(12):
        data_name.append(f'M{i+1}SCV{j+1}')

df = pd.DataFrame(data, columns=data_name)
df.head(5)

,Age,Sex,Age_Sex,M1SCV1,M1SCV2,M1SCV3,M1SCV4,M1SCV5,M1SCV6,M1SCV7,...,M2SCV3,M2SCV4,M2SCV5,M2SCV6,M2SCV7,M2SCV8,M2SCV9,M2SCV10,M2SCV11,M2SCV12
0,52.0,1.0,52.0,-1.795089,-0.055671,0.433509,-0.864642,-2.528412,1.646375,1.245978,...,0.002149,-0.345554,-1.842927,4.631274,-1.486365,0.415511,1.392088,0.155413,-3.563111,-0.496786
1,58.0,0.0,0.0,1.373096,-1.547226,-3.406749,-1.401614,1.392225,2.853063,-0.414824,...,-0.041056,0.498236,-0.815042,-2.177782,0.458931,-3.930059,1.000236,-1.055496,-0.075049,-0.799458
2,63.0,1.0,63.0,0.607958,1.490295,-0.188354,4.542142,-1.592139,2.927567,-1.447659,...,3.742965,-0.520675,-0.687089,-0.275345,-0.594670,0.064462,1.445954,0.461051,0.459807,1.523856
3,68.0,1.0,68.0,-0.952287,0.946707,-1.424925,-2.439162,0.450618,-0.446229,1.350085,...,-1.195516,-1.691283,1.811647,-0.207350,0.808894,1.351302,3.982187,-1.806670,-0.347542,1.666577
4,67.0,1.0,67.0,1.699567,1.457882,0.992004,0.625744,-1.692990,6.151593,1.029293,...,0.477786,1.442626,1.160964,-1.435246,-0.414841,-1.998198,-1.305428,-0.332297,-2.893012,0.132282


In [7]:
# evaluate first subspace 
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV1 + M1SCV2 + M2SCV1 + M2SCV2', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0066 3.0000 2900.0000 145984.3032 0.0000
         Pillai's trace   0.9934 3.0000 2900.0000 145984.3032 0.0000
 Hotelling-Lawley trace 151.0182 3.0000 2900.0000 145984.3032 0.0000
    Roy's greatest root 151.0182 3.0000 2900.0000 145984.3032 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
           M1SCV1          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9242  3.0000  2900.0000 

In [8]:
# evaluate second subspace 
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV3 + M1SCV4 + M2SCV3 + M2SCV4', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0064 3.0000 2900.0000 149782.6210 0.0000
         Pillai's trace   0.9936 3.0000 2900.0000 149782.6210 0.0000
 Hotelling-Lawley trace 154.9475 3.0000 2900.0000 149782.6210 0.0000
    Roy's greatest root 154.9475 3.0000 2900.0000 149782.6210 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
           M1SCV3          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9938  3.0000  2900.0000 

In [9]:
# evaluate third subspace 
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV5 + M1SCV6 + M2SCV5 + M2SCV6', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0067 3.0000 2900.0000 144198.4093 0.0000
         Pillai's trace   0.9933 3.0000 2900.0000 144198.4093 0.0000
 Hotelling-Lawley trace 149.1708 3.0000 2900.0000 144198.4093 0.0000
    Roy's greatest root 149.1708 3.0000 2900.0000 144198.4093 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M1SCV5          Value   Num DF    Den DF   F Value   Pr > F
---------------------------------------------------------------------
           Wilks' lambda  0.7672  3.0000  2900.0000  

In [10]:
# evaluate forth subspace 
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV7 + M1SCV8 + M2SCV7 + M2SCV8', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0068 3.0000 2900.0000 141252.8475 0.0000
         Pillai's trace   0.9932 3.0000 2900.0000 141252.8475 0.0000
 Hotelling-Lawley trace 146.1236 3.0000 2900.0000 141252.8475 0.0000
    Roy's greatest root 146.1236 3.0000 2900.0000 141252.8475 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
           M1SCV7          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9932  3.0000  2900.0000 

In [11]:
# evaluate fifth subspace 
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV9 + M1SCV10 + M2SCV9 + M2SCV10', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0063 3.0000 2900.0000 153563.5713 0.0000
         Pillai's trace   0.9937 3.0000 2900.0000 153563.5713 0.0000
 Hotelling-Lawley trace 158.8589 3.0000 2900.0000 153563.5713 0.0000
    Roy's greatest root 158.8589 3.0000 2900.0000 153563.5713 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M1SCV9          Value   Num DF    Den DF   F Value   Pr > F
---------------------------------------------------------------------
           Wilks' lambda  0.8897  3.0000  2900.0000  

In [12]:
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV11', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0067 3.0000 2903.0000 144472.8811 0.0000
         Pillai's trace   0.9933 3.0000 2903.0000 144472.8811 0.0000
 Hotelling-Lawley trace 149.3003 3.0000 2903.0000 144472.8811 0.0000
    Roy's greatest root 149.3003 3.0000 2903.0000 144472.8811 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M1SCV11          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9331  3.0000  2903.0000 

In [14]:
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M1SCV12', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0068 3.0000 2903.0000 140335.0545 0.0000
         Pillai's trace   0.9932 3.0000 2903.0000 140335.0545 0.0000
 Hotelling-Lawley trace 145.0242 3.0000 2903.0000 140335.0545 0.0000
    Roy's greatest root 145.0242 3.0000 2903.0000 140335.0545 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M1SCV12          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9858  3.0000  2903.0000 

In [15]:
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M2SCV11', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0069 3.0000 2903.0000 139037.7573 0.0000
         Pillai's trace   0.9931 3.0000 2903.0000 139037.7573 0.0000
 Hotelling-Lawley trace 143.6835 3.0000 2903.0000 139037.7573 0.0000
    Roy's greatest root 143.6835 3.0000 2903.0000 139037.7573 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M2SCV11          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9949  3.0000  2903.0000 

In [16]:
fit = MANOVA.from_formula('Age + Sex + Age_Sex ~ M2SCV12', data=df)
print(fit.mv_test())

                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda   0.0069 3.0000 2903.0000 139779.9389 0.0000
         Pillai's trace   0.9931 3.0000 2903.0000 139779.9389 0.0000
 Hotelling-Lawley trace 144.4505 3.0000 2903.0000 139779.9389 0.0000
    Roy's greatest root 144.4505 3.0000 2903.0000 139779.9389 0.0000
--------------------------------------------------------------------
                                                                    
---------------------------------------------------------------------
          M2SCV12          Value   Num DF    Den DF   F Value  Pr > F
---------------------------------------------------------------------
            Wilks' lambda  0.9867  3.0000  2903.0000 